In [1]:
import dtaidistance as dtai
import rust_dtw
import numpy as np
%load_ext snakeviz

In [2]:
from nilearn import datasets

development_dataset = datasets.fetch_development_fmri()

/home/fleetwood/miniconda3/envs/dtw/lib/python3.6/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [3]:
from nilearn.input_data import NiftiMapsMasker

msdl_data = datasets.fetch_atlas_msdl()
msdl_coords = msdl_data.region_coords

masker = NiftiMapsMasker(
    msdl_data.maps, resampling_target="data", t_r=2, detrend=True,
    low_pass=.1, high_pass=.01, memory='nilearn_cache', memory_level=1).fit()
masked_data = [masker.transform(func, confounds) for
               (func, confounds) in zip(
                   development_dataset.func, development_dataset.confounds)]

/home/fleetwood/miniconda3/envs/dtw/lib/python3.6/site-packages/numpy/lib/npyio.py:2349: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [4]:
from sklearn.preprocessing import StandardScaler
#normalizing to zero mean and unit std as per 
normalized = []
for masked in masked_data:
    normalized.append(StandardScaler().fit_transform(masked))

normalized=np.array(normalized)

In [5]:
#Now we need to accurately time how long it takes for each of them to compute the matricies
def compute_rust_dtw(timeseries):
    #implement rust_dtw
    connectomes = rust_dtw.dtw_connectomes(
        connectomes=timeseries, 
        window=1200, 
        vectorize=False, 
        distance_mode="euclidean"
    )
    return connectomes


def compute_dtai(timeseries):
    #implement rust_dtw
    connectomes = []
    for ts in timeseries:
        ds = dtai.dtw.distance_matrix_fast(ts.T, compact=False, use_mp=True)
        connectomes.append(ds)
    return connectomes

In [6]:
rust_dtw_result = compute_rust_dtw(normalized)
dtai_result = compute_dtai(normalized)

np.testing.assert_almost_equal(rust_dtw_result, dtai_result)

In [7]:
%snakeviz compute_rust_dtw(normalized)

 
*** Profile stats marshalled to file '/tmp/tmpn3r74e92'. 
Embedding SnakeViz in this document...


In [8]:
%snakeviz compute_dtai(normalized)

 
*** Profile stats marshalled to file '/tmp/tmpu4wnvyz2'. 
Embedding SnakeViz in this document...


As we can see, we achieve a 5x speedup compared to DTAIDistance Python OpenMP implementation. If you can get the C version to compile (I certainly can't) please submit a PR.